diag 1 ya fq -> gt
diag 2 sensorial -> fqq
diag 3 sens -> ingr  (fq?)

preguntas:
sensoriales en tablas son agregaciones cómo?

In [1]:
import re
from collections import defaultdict
from pathlib import Path

import numpy as np
import pandas as pd

FILE_PATH = r"D:\python-gcpds.luker_multiple_annotators\data\formulacion_y_sensorial.xlsx"

In [12]:
file = pd.ExcelFile(FILE_PATH)

In [81]:
df_form = pd.read_excel(FILE_PATH, sheet_name=0)
df_form.columns = ["codigo", "ingrediente", "mezcla"]
df_form.ffill(inplace=True)
df_form = df_form.pivot_table(index="codigo", columns="ingrediente", values="mezcla")
def _clean_colname(c):
    c = str(c).replace("\n", "/")
    c = re.sub(r"\s+", " ", c).strip()
    c = c.replace(" /", "/").replace("/ ", "/")
    return c
df_form = df_form.rename(columns=_clean_colname)

In [82]:
# parsing codes
def _parse_codes(colname: str):
    # Primary rule: exact 7-digit codes
    codes = re.findall(r"\d{7}", colname)
    return list(dict.fromkeys(codes))  # dedupe, keep order
cols_with_codes = {col: _parse_codes(col) for col in df_form.columns}

# build expanded per-code
expanded_parts = []
for col, codes in cols_with_codes.items():
    for code in codes:
        s = df_form[col].copy()
        s.name = code
        expanded_parts.append(s)
df_codes_expanded = pd.concat(expanded_parts, axis=1)

# If the same code appeared in multiple source columns, coalesce row-wise
unique_codes = pd.Index(pd.unique(df_codes_expanded.columns))
coalesced_cols = []
for code in unique_codes:
    sub = df_codes_expanded.loc[:, df_codes_expanded.columns == code]
    # Optional: detect conflicts (rows where two non-null values disagree)
    # Only meaningful if values are not all identical per row
    try:
        conflict_mask = sub.nunique(axis=1, dropna=True) > 1
        if conflict_mask.any():
            # You can inspect these rows if needed:
            print(f"[WARN] Code {code}: {conflict_mask.sum()} rows have conflicting values; taking first non-null.")
            pass
    except Exception:
        # Non-comparable types; just skip conflict check
        pass

    # Coalesce left-to-right: first non-null across duplicates
    merged_series = sub.bfill(axis=1).iloc[:, 0]
    merged_series.name = code
    coalesced_cols.append(merged_series)
df_codes_expanded = pd.concat(coalesced_cols, axis=1)

In [83]:
class DSU:
    def __init__(self):
        self.p = {}
        self.r = {}
    def _add(self, x):
        if x not in self.p:
            self.p[x] = x
            self.r[x] = 0
    def find(self, x):
        self._add(x)
        if self.p[x] != x:
            self.p[x] = self.find(self.p[x])
        return self.p[x]
    def union(self, a, b):
        ra, rb = self.find(a), self.find(b)
        if ra == rb: return
        if self.r[ra] < self.r[rb]:
            self.p[ra] = rb
        elif self.r[ra] > self.r[rb]:
            self.p[rb] = ra
        else:
            self.p[rb] = ra
            self.r[ra] += 1

dsu = DSU()
for _, codes in cols_with_codes.items():
    # union all codes that appeared together in the same header
    for i in range(len(codes) - 1):
        dsu.union(codes[i], codes[i+1])

# Materialize groups
groups = defaultdict(list)
for code in unique_codes:
    groups[dsu.find(code)].append(code)

In [84]:
# Sort members in each group (as integers if possible)
def _sort_codes(lst):
    try:
        return sorted(lst, key=lambda x: int(x))
    except Exception:
        return sorted(lst)

equivalence_groups = {root: _sort_codes(members) for root, members in groups.items()}

# ----------------------------
# 5) Merge columns by equivalence group
#    Each group collapses to a single canonical column (smallest code)
# ----------------------------
merged_cols = []
merged_names = []
for root, members in equivalence_groups.items():
    # only keep members that actually exist in df (defensive)
    present = [m for m in members if m in df_codes_expanded.columns]
    if not present:
        continue

    block = df_codes_expanded[present]
    # Coalesce across the group's columns, row-wise
    s = block.bfill(axis=1).iloc[:, 0]
    # Canonical column name = smallest code in the group
    canonical = _sort_codes(present)[0]
    s.name = canonical
    merged_cols.append(s)
    merged_names.append(canonical)

df_equiv_merged = pd.concat(merged_cols, axis=1) if merged_cols else pd.DataFrame(index=df_form.index)

In [85]:
{k: v for k, v in equivalence_groups.items() if len(v) > 1}

{'2000021': ['2000021', '2000302'],
 '2000696': ['2000023', '2000696'],
 '2000025': ['2000025', '2000829'],
 '2000614': ['2000026',
  '2000031',
  '2000606',
  '2000614',
  '2000639',
  '2000668',
  '2000684',
  '2000685',
  '2000831',
  '2000833',
  '2000835',
  '2001117'],
 '2000889': ['2000889', '2000970', '2000971'],
 '4100107': ['4100107', '4102615'],
 '4101441': ['4101441', '4102338']}

In [86]:
df_equiv_merged

,2000018,2000020,2000021,2000023,2000024,2000025,2000027,2000026,2000036,2000037,...,4101920,4102148,4102402,4102686,4103026,4110022,4200321,4500322,4500358,4500359
codigo,,,,,,,,,,,,,,,,,,,,,
PNM1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PNM10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PNM100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.0,NaN,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PNM101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PNM102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PNM95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,25.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PNM96,NaN,NaN,NaN,NaN,NaN,75.0,NaN,NaN,NaN,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PNM97,NaN,NaN,NaN,60.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
df_equiv_merged.loc[df_equiv_merged.sum(axis=1) > 100].iloc[0].dropna()

2000052    12.300
4100006    27.100
4100713     0.150
4100760    50.025
4100775     0.150
4100830     0.250
4101774    60.000
Name: PNM1, dtype: float64